In [6]:
import aiohttp
import http
from bs4 import BeautifulSoup
import asyncio
import requests
import pandas as pd
import re, time, os

FUNCTIONS

In [25]:
def get_url(url, headers=None, params=None):
    
    response = requests.get(url,headers=headers, params=params)
    stat = response.status_code
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(1)
    
    return stat, soup

def fetch_dict(soup, gameid):
    k_value = 0
    players = soup.find("tbody")
    play = re.split(r"[\n\t]+",players.get_text().strip())
    player_game = {} 

    for i in range(0, len(play), 5):
        # Date
        date = play[i]
        # Peak players
        peak = play[i+1]  
        # Average players
        average = play[i +4]
        player_game[k_value]={"Date": date,
                            "Peak Players":peak,
                            "Avg Players":average,
        }
        
        k_value += 1
        
    return player_game

def overall_fetch(list_id):
    error_list =[]
    game_id_dict = {}
    for gameid in list_id:
        gameid = str(gameid)
        url = f'https://steamcharts.com/app/{gameid}'
        headers= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
        try:
            stat, soup = get_url(url, headers=headers, params=gameid)
            if stat == 200:
                print(f"Webcode status is : {stat} for {gameid} id. ")
                player_game = fetch_dict(soup, gameid)
                game_id_dict[gameid] = player_game.copy()
            else:
                error_list.append((gameid, stat))
                print(f"Appid: {gameid} reported an error {stat}, skipping it for now")    
        except Exception as e:  # Catch specific exceptions
            error_list.append((gameid, stat))
            print(f"App ID: {gameid} reported an error: {str(e)}, skipping it for now.")
            
    
    return game_id_dict, error_list

MAIN CODES

In [14]:
# creates the id list
txt = pd.read_csv(r"C:\\Users\\Utilizador\\Desktop\\IRONHACK\\Project 3\\Project-3\\top1k.csv")
txt = txt.rename({"Game Title":"title", "Steam App ID":"id"}, axis=1)
list_id = list(txt["id"])

In [ ]:
## main program, output

game_id_dict, error_list = overall_fetch(list_id)


In [22]:
output = pd.DataFrame.from_dict(game_id_dict)
output.to_feather("output.feather")

In [24]:
err_table = pd.DataFrame(error_list)
err_table.to_csv("error.csv")


In [26]:
output

,578080,2358720,1623730,730,1599340,570,1091500,1245620,2923300,1063730,...,1599600,1611910,2142790,1238860,803330,2495450,2103140,805550,1062090,1874880
0,"{'Date': 'Last 30 Days', 'Peak Players': '2946...","{'Date': 'Last 30 Days', 'Peak Players': '4847...","{'Date': 'Last 30 Days', 'Peak Players': '1929...","{'Date': 'Last 30 Days', 'Peak Players': '8559...","{'Date': 'Last 30 Days', 'Peak Players': '1769...","{'Date': 'Last 30 Days', 'Peak Players': '4295...","{'Date': 'Last 30 Days', 'Peak Players': '2380...","{'Date': 'Last 30 Days', 'Peak Players': '4632...","{'Date': 'Last 30 Days', 'Peak Players': '3812...","{'Date': 'Last 30 Days', 'Peak Players': '3871...",...,"{'Date': 'Last 30 Days', 'Peak Players': '1596...","{'Date': 'Last 30 Days', 'Peak Players': '392....","{'Date': 'Last 30 Days', 'Peak Players': '3982...","{'Date': 'Last 30 Days', 'Peak Players': '977....","{'Date': 'Last 30 Days', 'Peak Players': '52.2...","{'Date': 'Last 30 Days', 'Peak Players': '364....","{'Date': 'Last 30 Days', 'Peak Players': '401....","{'Date': 'Last 30 Days', 'Peak Players': '2205...","{'Date': 'Last 30 Days', 'Peak Players': '2744...","{'Date': 'Last 30 Days', 'Peak Players': '2243..."
1,"{'Date': 'August 2024', 'Peak Players': '28524...","{'Date': 'August 2024', 'Peak Players': '97044...","{'Date': 'August 2024', 'Peak Players': '26179...","{'Date': 'August 2024', 'Peak Players': '89733...","{'Date': 'August 2024', 'Peak Players': '19985...","{'Date': 'August 2024', 'Peak Players': '44979...","{'Date': 'August 2024', 'Peak Players': '24479...","{'Date': 'August 2024', 'Peak Players': '80731...","{'Date': 'August 2024', 'Peak Players': '40683...","{'Date': 'August 2024', 'Peak Players': '3285....",...,"{'Date': 'August 2024', 'Peak Players': '2036....","{'Date': 'August 2024', 'Peak Players': '386.3...","{'Date': 'August 2024', 'Peak Players': '6017....","{'Date': 'August 2024', 'Peak Players': '1180....","{'Date': 'August 2024', 'Peak Players': '183.1...","{'Date': 'August 2024', 'Peak Players': '413.4...","{'Date': 'August 2024', 'Peak Players': '400.4...","{'Date': 'August 2024', 'Peak Players': '2232....","{'Date': 'August 2024', 'Peak Players': '3145....","{'Date': 'August 2024', 'Peak Players': '2053...."
2,"{'Date': 'July 2024', 'Peak Players': '293396....",NaN,"{'Date': 'July 2024', 'Peak Players': '53788.3...","{'Date': 'July 2024', 'Peak Players': '875365....","{'Date': 'July 2024', 'Peak Players': '19612.2...","{'Date': 'July 2024', 'Peak Players': '475393....","{'Date': 'July 2024', 'Peak Players': '24961.6...","{'Date': 'July 2024', 'Peak Players': '209287....","{'Date': 'July 2024', 'Peak Players': '369328....","{'Date': 'July 2024', 'Peak Players': '3640.92...",...,"{'Date': 'July 2024', 'Peak Players': '1663.21...","{'Date': 'July 2024', 'Peak Players': '342.48'...",NaN,"{'Date': 'July 2024', 'Peak Players': '1428.65...","{'Date': 'July 2024', 'Peak Players': '43.12',...","{'Date': 'July 2024', 'Peak Players': '639.88'...","{'Date': 'July 2024', 'Peak Players': '640.55'...","{'Date': 'July 2024', 'Peak Players': '2498.67...","{'Date': 'July 2024', 'Peak Players': '3570.91...","{'Date': 'July 2024', 'Peak Players': '2259.52..."
3,"{'Date': 'June 2024', 'Peak Players': '287498....",NaN,"{'Date': 'June 2024', 'Peak Players': '26105.0...","{'Date': 'June 2024', 'Peak Players': '947520....","{'Date': 'June 2024', 'Peak Players': '19317.9...","{'Date': 'June 2024', 'Peak Players': '486674....","{'Date': 'June 2024', 'Peak Players': '21812.7...","{'Date': 'June 2024', 'Peak Players': '219194....","{'Date': 'June 2024', 'Peak Players': '399467....","{'Date': 'June 2024', 'Peak Players': '5073.51...",...,"{'Date': 'June 2024', 'Peak Players': '1552.03...","{'Date': 'June 2024', 'Peak Players': '401.08'...",NaN,"{'Date': 'June 2024', 'Peak Players': '905.44'...","{'Date': 'June 2024', 'Peak Players': '42.23',...","{'Date': 'June 2024', 'Peak Players': '453.47'...","{'Date': 'June 2024', 'Peak Players': '653.15'...","{'Date': 